In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import time

# 检查是否有可用的GPU
if torch.cuda.is_available():
    # 设置第二块GPU设备
    device = torch.device("cuda:1")  # "cuda:1" 表示第二块GPU
else:
    device = torch.device("cpu")  # 如果没有可用的GPU，则使用CPU

print(device)
num_workers=16
start_time = time.time()
# 定义前馈神经网络模型
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.LeakyReLU(negative_slope=0.01)
        
        self.fc2 = nn.Linear(hidden_size, 128)
        self.relu2 = nn.LeakyReLU(negative_slope=0.01)
        
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# 定义超参数
input_size = 28 * 28  # 输入特征的维度
hidden_size = 512  # 隐藏层的大小
num_classes = 10  # 类别的数量
learning_rate = 0.001  # 学习率
num_epochs = 10  # 迭代次数
batch_size = 64  # 批次大小

# 加载数据集
train_dataset = MNIST(root='~/Datasets/MNIST', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='~/Datasets/MNIST', train=False, transform=ToTensor(), download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers)

# 初始化模型
model = FeedforwardNN(input_size, hidden_size, num_classes).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.view(-1, input_size).to(device)
        targets=targets.to(device)
        # 前向传播
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


    # 在测试集上评估模型
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            data = data.view(-1, input_size).to(device)
            targets=targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')
            
end_time = time.time()
training_time = end_time - start_time
print("模型的training_time为"+str(round(training_time,2))+"秒" )

cuda:1
Epoch [1/10], Step [100/938], Loss: 0.3262
Epoch [1/10], Step [200/938], Loss: 0.3551
Epoch [1/10], Step [300/938], Loss: 0.2663
Epoch [1/10], Step [400/938], Loss: 0.3614
Epoch [1/10], Step [500/938], Loss: 0.1497
Epoch [1/10], Step [600/938], Loss: 0.1832
Epoch [1/10], Step [700/938], Loss: 0.0552
Epoch [1/10], Step [800/938], Loss: 0.0453
Epoch [1/10], Step [900/938], Loss: 0.1586
Test Accuracy: 0.9625
Epoch [2/10], Step [100/938], Loss: 0.1082
Epoch [2/10], Step [200/938], Loss: 0.1522
Epoch [2/10], Step [300/938], Loss: 0.2218
Epoch [2/10], Step [400/938], Loss: 0.2023
Epoch [2/10], Step [500/938], Loss: 0.0990
Epoch [2/10], Step [600/938], Loss: 0.0718
Epoch [2/10], Step [700/938], Loss: 0.0749
Epoch [2/10], Step [800/938], Loss: 0.0214
Epoch [2/10], Step [900/938], Loss: 0.0633
Test Accuracy: 0.9740
Epoch [3/10], Step [100/938], Loss: 0.0117
Epoch [3/10], Step [200/938], Loss: 0.0330
Epoch [3/10], Step [300/938], Loss: 0.0290
Epoch [3/10], Step [400/938], Loss: 0.0249
Epo